# Save data to file

In [1]:
import numpy as np
import os
import geopandas
import pandas as pd

In [2]:
from stroke_maps.catchment import Catchment
from stroke_maps.combine import Combine
import stroke_maps.geo
import stroke_maps.plot
from stroke_maps.utils import convert_df_to_gdf

combine = Combine()

## Setup for scenarios

In [3]:
dir_output = os.path.join('output', 'run')

# Temporary rubbish dir extender to prevent overwriting:
while os.path.exists(dir_output):
    dir_output += '1'

os.mkdir(dir_output)

In [4]:
merge_dicts = dict(
    df_transfer = {},
    df_lsoa = {},
    df_units = {},
)

In [5]:
units_to_include = [
    'GL13NN',
    'SN36BB',
    'BS105NB',
    'EX314JB',
    'PL68DH',
    'TR13LQ',
    'EX25DW',
    'BA13NG',
    'SP28BJ',
    'TA15DA',
    'TQ27AA',
    'BS234TQ',
    'BA214AT',
]

## Scenario 1

In [6]:
scenario_name = 'drip-and-ship'

In [7]:
# Catchment overwrites default values
catchment = Catchment({
    # "setup": setup,
    'name': scenario_name,
    "run_duration": 5,  # years
})

df_units = catchment.get_unit_services()

df_units = df_units.reset_index()

mask = df_units['postcode'].isin(units_to_include)
df_units.loc[mask, 'selected'] = 1

df_units = df_units.set_index('postcode')

In [8]:
df_units['stroke_team'][df_units['selected'] == 1]

postcode
BA13NG                     Royal United Hospital Bath
EX314JB     North Devon District Hospital, Barnstaple
PL68DH                   Derriford Hospital, Plymouth
EX25DW                Royal Devon and Exeter Hospital
GL13NN     Gloucestershire Royal Hospital, Gloucester
SN36BB                Great Western Hospital, Swindon
TA15DA                Musgrove Park Hospital, Taunton
BS105NB            North Bristol Hospital (Southmead)
SP28BJ                    Salisbury District Hospital
TQ27AA                                Torbay Hospital
TR13LQ                 Royal Cornwall Hospital, Truro
BS234TQ    Weston General Hospital, Weston-super-Mare
BA214AT                      Yeovil District Hospital
Name: stroke_team, dtype: object

In [9]:
dict_useful_data = catchment.main(df_units)

Gather data for later combination:

In [10]:
# Update big dictionary of Dataframes:
merge_dicts['df_units'][scenario_name] = dict_useful_data['df_units']
merge_dicts['df_transfer'][scenario_name] = dict_useful_data['df_transfer']
merge_dicts['df_lsoa'][scenario_name] = dict_useful_data['df_lsoa']

Save data to file:

In [11]:
dir_output_scenario = os.path.join(dir_output, scenario_name)

if os.path.exists(dir_output_scenario):
    pass
else:
    os.mkdir(dir_output_scenario)

In [12]:
file_dict = {
    'df_transfer': {
        'file': 'df_transfer.csv',
        'data': dict_useful_data['df_transfer']
    },
    'df_units': {
        'file': 'df_units.csv',
        'data': merge_dicts['df_units'][scenario_name]
    },
    'df_lsoa': {
        'file': 'df_lsoa.csv',
        'data': merge_dicts['df_lsoa'][scenario_name]
    }
}

In [13]:
for data_type, data_dict in file_dict.items():
    path_to_file = os.path.join(dir_output_scenario, data_dict['file'])
    data_dict['data'].to_csv(path_to_file)

## Scenario 2

In [14]:
scenario_name = 'mothership'

In [15]:
# Catchment overwrites default values
catchment = Catchment({
    # "setup": setup,
    'name': scenario_name,
    "run_duration": 5,  # years
})

df_units = catchment.get_unit_services()

df_units = df_units.reset_index()

# Set all non-MT units to have no IVT:
mask = df_units['use_mt'] == 0
df_units.loc[mask, 'use_ivt'] = 0

mask = (
    (df_units['postcode'].isin(units_to_include)) &
    (df_units['use_ivt'] == 1)
)
df_units.loc[mask, 'selected'] = 1


df_units = df_units.set_index('postcode')

In [16]:
df_units['stroke_team'][df_units['selected'] == 1]

postcode
PL68DH           Derriford Hospital, Plymouth
BS105NB    North Bristol Hospital (Southmead)
Name: stroke_team, dtype: object

In [ ]:
dict_useful_data = catchment.main(df_units)

Gather data for later combination:

In [ ]:
# Update big dictionary of Dataframes:
merge_dicts['df_units'][scenario_name] = dict_useful_data['df_units']
merge_dicts['df_transfer'][scenario_name] = dict_useful_data['df_transfer']
merge_dicts['df_lsoa'][scenario_name] = dict_useful_data['df_lsoa']

In [ ]:
dict_useful_data['df_lsoa']

Save data to file:

In [ ]:
dir_output_scenario = os.path.join(dir_output, scenario_name)

if os.path.exists(dir_output_scenario):
    pass
else:
    os.mkdir(dir_output_scenario)

In [ ]:
file_dict = {
    'df_transfer': {
        'file': 'df_transfer.csv',
        'data': dict_useful_data['df_transfer']
    },
    'df_units': {
        'file': 'df_units.csv',
        'data': merge_dicts['df_units'][scenario_name]
    },
    'df_lsoa': {
        'file': 'df_lsoa.csv',
        'data': merge_dicts['df_lsoa'][scenario_name]
    }
}

In [ ]:
for data_type, data_dict in file_dict.items():
    path_to_file = os.path.join(dir_output_scenario, data_dict['file'])
    data_dict['data'].to_csv(path_to_file)

## Combine multiple scenarios

In [ ]:
combo_df_units = combine.combine_units(
    merge_dicts['df_units'])

combo_df_transfer = combine.combine_transfer(
    merge_dicts['df_transfer'])

combo_df_lsoa = combine.combine_lsoa(
    merge_dicts['df_lsoa'])

Save data to file:

In [ ]:
dir_output_scenario = os.path.join(dir_output, 'combined')

if os.path.exists(dir_output_scenario):
    pass
else:
    os.mkdir(dir_output_scenario)

In [ ]:
file_dict = {
    'combo_df_units': {
        'file': 'df_units.csv',
        'data': combo_df_units,
    },
    'combo_df_transfer': {
        'file': 'df_transfer.csv',
        'data': combo_df_transfer,
    },
    'combo_df_lsoa': {
        'file': 'df_lsoa.csv',
        'data': combo_df_lsoa,
    }
}

In [ ]:
for data_type, data_dict in file_dict.items():
    path_to_file = os.path.join(dir_output_scenario, data_dict['file'])
    data_dict['data'].to_csv(path_to_file)

## Create geometry data

In [ ]:
dir_combined = os.path.join(dir_output, 'combined')

In [ ]:
# Load in the data

df_units = pd.read_csv(
    os.path.join(dir_combined, 'df_units.csv'),
    index_col=0,
    header=[0, 1, 2],
)
df_transfer = pd.read_csv(
    os.path.join(dir_combined, 'df_transfer.csv'),
    index_col=[0, 1],
    header=[0, 1],
)
df_lsoa = pd.read_csv(
    os.path.join(dir_combined, 'df_lsoa.csv'),
    index_col=[0, 1],
    header=[0, 1, 2],
)

In [ ]:
df_regions = stroke_maps.geo.load_regions()

In [ ]:
(
    gdf_boundaries_regions,
    gdf_points_units,
    gdf_lines_transfer,
    gdf_boundaries_lsoa,
    gdf_boundaries_catchment
) = stroke_maps.geo.main(
    combo_df_lsoa, combo_df_units, df_regions, combo_df_transfer
    )

Save data to file:

In [ ]:
# Create a new maps/ dir for outputs.
dir_output_maps = os.path.join(dir_combined, 'maps')
try:
    os.mkdir(dir_output_maps)
except FileExistsError:
    # The directory already exists.
    pass

In [ ]:
file_dict = {
    'gdf_boundaries_regions': {
        'file': 'gdf_boundaries_regions.csv',
        'data': gdf_boundaries_regions,
    },
    'gdf_points_units': {
        'file': 'gdf_points_units.csv',
        'data': gdf_points_units,
    },
    'gdf_lines_transfer': {
        'file': 'gdf_lines_transfer.csv',
        'data': gdf_lines_transfer,
    },
    'gdf_boundaries_lsoa': {
        'file': 'gdf_boundaries_lsoa.csv',
        'data': gdf_boundaries_lsoa, 
    },
    'gdf_boundaries_catchment': {
        'file': 'gdf_boundaries_catchment.csv',
        'data': gdf_boundaries_catchment,
    },
}

In [ ]:
for data_type, data_dict in file_dict.items():
    path_to_file = os.path.join(dir_output_maps, data_dict['file'])
    data_dict['data'].to_csv(path_to_file)

## Setup for maps

Crop all geometry data to the same axis limits and assign colours to the catchment areas:

In [ ]:
# Load in the data

gdf_boundaries_regions = pd.read_csv(
    os.path.join(dir_output_maps, 'gdf_boundaries_regions.csv'),
    index_col=[0],
    header=[0, 1],
)
gdf_points_units = pd.read_csv(
    os.path.join(dir_output_maps, 'gdf_points_units.csv'),
    index_col=[0],
    header=[0, 1],
)
gdf_lines_transfer = pd.read_csv(
    os.path.join(dir_output_maps, 'gdf_lines_transfer.csv'),
    index_col=[0],
    header=[0, 1],
)
gdf_boundaries_lsoa = pd.read_csv(
    os.path.join(dir_output_maps, 'gdf_boundaries_lsoa.csv'),
    index_col=[0],
    header=[0, 1, 2],
)
gdf_boundaries_catchment = pd.read_csv(
    os.path.join(dir_output_maps, 'gdf_boundaries_catchment.csv'),
    index_col=[0],
    header=[0, 1],
)

In [ ]:
gdf_boundaries_regions = convert_df_to_gdf(gdf_boundaries_regions)
gdf_points_units = convert_df_to_gdf(gdf_points_units)
gdf_lines_transfer = convert_df_to_gdf(gdf_lines_transfer)
gdf_boundaries_lsoa = convert_df_to_gdf(gdf_boundaries_lsoa)
gdf_boundaries_catchment = convert_df_to_gdf(gdf_boundaries_catchment)

In [ ]:
(
gdf_boundaries_regions,
gdf_points_units,
gdf_boundaries_catchment,
gdf_boundaries_lsoa,
gdf_lines_transfer,
box_shared,
map_extent_shared
        ) = stroke_maps.plot.main(
    gdf_boundaries_regions,
    gdf_points_units,
    gdf_boundaries_catchment,
    gdf_boundaries_lsoa,
    gdf_lines_transfer,
    crop_axis_leeway=5000
    )

## Plotting

Combining the scenarios earlier means that these maps share axis limits even though they contain different information for each scenario.

In [ ]:
for scenario in ['drip-and-ship', 'mothership']:
    stroke_maps.plot.plot_map_selected_regions(
        gdf_boundaries_regions,
        gdf_points_units,
        scenario=scenario,
        map_extent=map_extent_shared,
        path_to_file=os.path.join(dir_output_maps, f'map_regions_{scenario}.png')
    )

In [ ]:
for scenario in ['drip-and-ship', 'mothership']:
    stroke_maps.plot.plot_map_selected_units(
        gdf_boundaries_regions,
        gdf_points_units,
        gdf_lines_transfer,
        scenario=scenario,
        map_extent=map_extent_shared,
        path_to_file=os.path.join(dir_output_maps, f'map_units_{scenario}.png')
    )

In [ ]:
for scenario in ['drip-and-ship', 'mothership']:
    stroke_maps.plot.plot_map_catchment(
        gdf_boundaries_catchment,
        gdf_boundaries_regions,
        gdf_points_units,
        gdf_lines_transfer,
        scenario=scenario,
        map_extent=map_extent_shared,
        path_to_file=os.path.join(dir_output_maps, f'map_catchment_{scenario}.png')
    )